# About
### this notebook's goal is to do some basic EDA to get idea of how the data look like

# Main tasks: 
 - Sečíst přítoky a odečíst odtoky z ČR
 - Porovnat letošní rok oproti minulými - jsme na tom lépe nebo hůře?
    - co kdybychom vypli flow in? - Dívat se na celoevropská data


# Sub tasks:
 - Zjistit co znamená reálně firm, ověřit jestli je tam nějaký limit na denní odběr třeba kvůli nějakým fyzikálním vlastnostem
 - vliv teploty 
# Data
### data is availible in folder data, in 4 subfolders:
 - flow_in are data about gas flowing in cz
 - flow_out data about gas flowing out of cz
 - storage_in - from CZ to storage
 - storage_out - from storage to CZ

### all xlsx files have only ONE sheet
 


In [46]:
# Imports:
%load_ext autoreload
%autoreload 2
from tqdm import tqdm
import time
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.ticker as plticker
from datetime import datetime
import plotly.graph_objects as go
import copy
from plotly.subplots import make_subplots

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def create_dict_datasets(data_folder: str = '../data') -> dict:
    ''' 
    Loads all datasets into dictionaries
    with names same as data folders in 
    :param data_folder: folder where all dataset folders are saved
    '''
    dict_of_all = {}
    #get all dataset folders
    datasets = os.listdir(data_folder)
    #remove hidden files
    dict_of_all = {dataset:0 for dataset in datasets  if not dataset.startswith('.')}
    print(f'initiating for dataset groups: {list(dict_of_all.keys())}')
    for dataset in dict_of_all.keys():
        print(f'loading data for dataset group: {dataset}')
        # list all files in dataset folder
        files_in_dataset = os.listdir(f'{data_folder}/{dataset}')
        # initiate dictionary
        files_in_dataset = {k:0 for k in files_in_dataset}
        dict_of_all[dataset] = files_in_dataset
        for file in dict_of_all[dataset].keys():
            # assign dataframe to each filename
            dict_of_all[dataset][file] = pd.read_excel(f'{data_folder}/{dataset}/{file}')
    return dict_of_all
            


In [ ]:
dict_of_all = create_dict_datasets()

In [ ]:
dict_of_all['storage_out'].keys()
dict_of_all['storage_in'].keys()
dict_of_all['flow_out'].keys()
dict_of_all['flow_in'].keys()

In [ ]:
#get first dataframe from each dataset folder
storage_out_east_to_cz = dict_of_all['storage_out'][list(dict_of_all['storage_out'].keys())[0]]
storage_in_cz_to_center = dict_of_all['storage_in'][list(dict_of_all['storage_in'].keys())[0]]
flow_in_germany_north_the_lowest_to_cz = dict_of_all['flow_in'][list(dict_of_all['flow_in'].keys())[0]]
flow_out_cz_to_germany_barandov_opal_most_norther = dict_of_all['flow_out'][list(dict_of_all['flow_out'].keys())[0]]

In [ ]:
flow_in_germany_north_the_lowest_to_cz.info()

In [ ]:
flow_out_cz_to_germany_barandov_opal_most_norther.info()

In [ ]:
storage_in_cz_to_center.info()

In [ ]:
storage_out_east_to_cz.info()

In [ ]:
indicator_groups = storage_out_east_to_cz['indicator'].unique()
print(f'This dataset can be divided by column indicator into 2 groups: {indicator_groups}')
print(f'group {indicator_groups[0]} has this amount of zeros in "value" column:')
storage_out_east_to_cz.loc[storage_out_east_to_cz.indicator == 'Firm Available'].loc[storage_out_east_to_cz['value'] == 0]['value'].nunique()
print(f'group {indicator_groups[1]}: has this amount of unique values for "value" column where its value is == 0:')
storage_out_east_to_cz.loc[storage_out_east_to_cz.indicator == 'Physical Flow'].loc[storage_out_east_to_cz['value'] == 0].nunique()

storage_out_east_to_cz['flowStatus'].unique()
storage_out_east_to_cz.loc[storage_out_east_to_cz.flowStatus == 'Confirmed']

In [31]:
def firm_flow(default_data):
    data = copy.deepcopy(default_data)
    firm_availible_data = data.loc[data.indicator == 'Firm Available']
    physical_flow_data = data.loc[data.indicator == 'Physical Flow']
    return firm_availible_data, physical_flow_data

In [33]:
def prepare_data(data):
    data['periodFrom']  = pd.to_datetime(data['periodFrom'])
    data['periodFrom'] = data['periodFrom'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))
    return data

In [34]:
def plot_values(default_data):
    data = copy.deepcopy(default_data)
    fig = plt.figure(figsize=[14,10])
    ax = plt.subplot(111)
    data = prepare_data(data)
    # l = ax.fill_between(data['obdobi'],data['UK4/UK3'])
    ax.scatter(data['periodFrom'],data['value'])

    # set the basic properties
    ax.set_xlabel('date')
    ax.set_ylabel('value (kWh/d)')
    ax.set_title('time series of stored amount of gas')
    # ax.tick_params(axis="x", labelsie=6) 
    start, end = ax.get_xlim()
    loc = plticker.MultipleLocator(base=50.0)
    ax.xaxis.set_major_locator(loc)
    fig.show()

In [53]:

def plotly_plot_values(default_data,graph_type = 'scatter'):
    data = prepare_data(default_data)
    fig = make_subplots(rows=1, cols=1)
    if graph_type == 'scatter':
        fig.add_trace(go.Scatter(x = data['periodFrom'], y = data['value']),row=1,col=1)
    elif graph_type == 'hist':
        fig.add_trace(go.Histogram(x = data['periodFrom'], y = data['value']),row=1,col=1)
    # elif graph_type == 'line':
        # fig = go.Scatter(x = data['periodFrom'], y = data['value'])
    fig.show()

In [54]:



def plotly_subplot_values(default_data1, default_data2,graph_type = 'scatter'):
    data1 = prepare_data(default_data1)
    data2 = prepare_data(default_data2)
    fig = make_subplots(rows=1, cols=2)
    if graph_type == 'scatter':

        fig.add_trace(
            go.Scatter(x = data1['periodFrom'], y = data1['value']), row=1, col=1)
        

        fig.add_trace(
            go.Scatter(x = data2['periodFrom'], y = data2['value']), row=1, col=2)
        
    elif graph_type == 'hist':
        
        fig.add_trace(
            go.Histogram(x = data1['periodFrom'], y = data1['value']), row=1, col=1)
        

        fig.add_trace(
            go.Histogram(x = data2['periodFrom'], y = data2['value']),  row=1, col=2)
        
    # elif graph_type == 'line':
        # fig = go.Scatter(x = data['periodFrom'], y = data['value'])
    fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
    fig.show()

In [55]:
firm_availible_data,physical_flow_data = firm_flow(storage_out_east_to_cz)

In [56]:
plotly_subplot_values(firm_availible_data,physical_flow_data)

In [57]:
flow_in_firm_availible, flow_in_physical_flow = firm_flow(flow_in_germany_north_the_lowest_to_cz)

In [70]:
flow_in_germany_north_the_lowest_to_cz

,indicator,periodType,periodFrom,periodTo,tsoEicCode,operatorLabel,pointLabel,tsoItemIdentifier,directionKey,unit,...,generalRemarks,value,lastUpdateDateTime,isUnlimited,flowStatus,isCamRelevant,isNA,isCmpRelevant,interruptionCalculationRemark,isArchived
0,Firm Available,day,2020-01-22 06:00,2040-01-01 06:00,21X000000001304L,NET4GAS,Deutschneudorf EUGAL Brandov,21Z0000000004839,entry,kWh/d,...,NaN,0,2020-05-31 09:42,no,NaN,no,NaN,NaN,NaN,NaN
1,Physical Flow,day,2020-01-21 06:00,2020-01-22 06:00,21X000000001304L,NET4GAS,Deutschneudorf EUGAL Brandov,21Z0000000004839,entry,kWh/d,...,NaN,610403499,2020-01-22 12:19,NaN,Provisional,yes,NaN,yes,NaN,0.0
2,Physical Flow,day,2020-01-22 06:00,2020-01-23 06:00,21X000000001304L,NET4GAS,Deutschneudorf EUGAL Brandov,21Z0000000004839,entry,kWh/d,...,NaN,615611363,2020-01-23 12:05,NaN,Provisional,yes,NaN,yes,NaN,0.0
3,Physical Flow,day,2020-01-23 06:00,2020-01-24 06:00,21X000000001304L,NET4GAS,Deutschneudorf EUGAL Brandov,21Z0000000004839,entry,kWh/d,...,NaN,610985123,2020-01-24 12:05,NaN,Provisional,yes,NaN,yes,NaN,0.0
4,Physical Flow,day,2020-01-24 06:00,2020-01-25 06:00,21X000000001304L,NET4GAS,Deutschneudorf EUGAL Brandov,21Z0000000004839,entry,kWh/d,...,NaN,611512284,2020-01-25 12:05,NaN,Provisional,yes,NaN,yes,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Physical Flow,day,2022-10-11 06:00,2022-10-12 06:00,21X000000001304L,NET4GAS,Deutschneudorf EUGAL Brandov,21Z0000000004839,entry,kWh/d,...,NaN,72004364,2022-10-12 12:16,NaN,Provisional,no,NaN,no,NaN,0.0
996,Physical Flow,day,2022-10-12 06:00,2022-10-13 06:00,21X000000001304L,NET4GAS,Deutschneudorf EUGAL Brandov,21Z0000000004839,entry,kWh/d,...,NaN,71995753,2022-10-14 02:29,NaN,Provisional,no,NaN,no,NaN,0.0
997,Physical Flow,day,2022-10-13 06:00,2022-10-14 06:00,21X000000001304L,NET4GAS,Deutschneudorf EUGAL Brandov,21Z0000000004839,entry,kWh/d,...,NaN,59995208,2022-10-14 22:36,NaN,Provisional,no,NaN,no,NaN,0.0
998,Physical Flow,day,2022-10-14 06:00,2022-10-15 06:00,21X000000001304L,NET4GAS,Deutschneudorf EUGAL Brandov,21Z0000000004839,entry,kWh/d,...,NaN,54998128,2022-10-15 15:49,NaN,Provisional,no,NaN,no,NaN,0.0


In [58]:
plotly_plot_values(flow_in_physical_flow, 'scatter')

In [71]:
flow_out_cz_to_germany_barandov_opal_most_norther['value'].unique()

array([0])

In [72]:
flow_out_cz_to_germany_barandov_opal_most_norther

,indicator,periodType,periodFrom,periodTo,tsoEicCode,operatorLabel,pointLabel,tsoItemIdentifier,directionKey,unit,...,generalRemarks,value,lastUpdateDateTime,isUnlimited,flowStatus,isCamRelevant,isNA,isCmpRelevant,interruptionCalculationRemark,isArchived
0,Firm Available,day,2013-10-16 06:00,2040-01-01 06:00,21X000000001304L,NET4GAS,Brandov-OPAL (DE),21Z000000000242V,exit,kWh/d,...,NaN,0,2020-04-15 13:44,no,NaN,yes,NaN,NaN,NaN,NaN
1,Physical Flow,day,2017-10-15 06:00,2017-10-16 06:00,21X000000001304L,NET4GAS,Brandov-OPAL (DE),21Z000000000242V,exit,kWh/d,...,NaN,0,2017-11-12 04:18,NaN,Confirmed,NaN,NaN,NaN,NaN,0.0
2,Physical Flow,day,2017-10-16 06:00,2017-10-17 06:00,21X000000001304L,NET4GAS,Brandov-OPAL (DE),21Z000000000242V,exit,kWh/d,...,NaN,0,2017-11-12 04:18,NaN,Confirmed,NaN,NaN,NaN,NaN,0.0
3,Physical Flow,day,2017-10-17 06:00,2017-10-18 06:00,21X000000001304L,NET4GAS,Brandov-OPAL (DE),21Z000000000242V,exit,kWh/d,...,NaN,0,2017-11-12 04:18,NaN,Confirmed,NaN,NaN,NaN,NaN,0.0
4,Physical Flow,day,2017-10-18 06:00,2017-10-19 06:00,21X000000001304L,NET4GAS,Brandov-OPAL (DE),21Z000000000242V,exit,kWh/d,...,NaN,0,2017-11-12 04:18,NaN,Confirmed,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,Physical Flow,day,2022-10-11 06:00,2022-10-12 06:00,21X000000001304L,NET4GAS,Brandov-OPAL (DE),21Z000000000242V,exit,kWh/d,...,NaN,0,2022-10-12 12:16,NaN,Provisional,no,NaN,no,NaN,0.0
1824,Physical Flow,day,2022-10-12 06:00,2022-10-13 06:00,21X000000001304L,NET4GAS,Brandov-OPAL (DE),21Z000000000242V,exit,kWh/d,...,NaN,0,2022-10-14 02:29,NaN,Provisional,no,NaN,no,NaN,0.0
1825,Physical Flow,day,2022-10-13 06:00,2022-10-14 06:00,21X000000001304L,NET4GAS,Brandov-OPAL (DE),21Z000000000242V,exit,kWh/d,...,NaN,0,2022-10-14 22:36,NaN,Provisional,no,NaN,no,NaN,0.0
1826,Physical Flow,day,2022-10-14 06:00,2022-10-15 06:00,21X000000001304L,NET4GAS,Brandov-OPAL (DE),21Z000000000242V,exit,kWh/d,...,NaN,0,2022-10-15 15:49,NaN,Provisional,no,NaN,no,NaN,0.0


In [65]:
# flow_out_cz_to_germany_barandov_opal_most_norther['value'].unique()
# this does not make sense to do...
# flow_out_firm_availible, flow_out_physical_flow = firm_flow(flow_out_cz_to_germany_barandov_opal_most_norther)
# plotly_plot_values(flow_out_physical_flow, 'scatter')

array([0])

In [66]:
storage_in_firm_availible,storage_out_physical_flow = firm_flow(storage_in_cz_to_center)

In [69]:
plotly_plot_values(storage_in_firm_availible, 'scatter')

In [29]:
# flow_in_germany_north_the_lowest_to_cz.info()
# flow_out_cz_to_germany_barandov_opal_most_norther.info()
# storage_in_cz_to_center.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   indicator                      1000 non-null   object 
 1   periodType                     1000 non-null   object 
 2   periodFrom                     1000 non-null   object 
 3   periodTo                       1000 non-null   object 
 4   tsoEicCode                     1000 non-null   object 
 5   operatorLabel                  1000 non-null   object 
 6   pointLabel                     1000 non-null   object 
 7   tsoItemIdentifier              1000 non-null   object 
 8   directionKey                   1000 non-null   object 
 9   unit                           1000 non-null   object 
 10  itemRemarks                    0 non-null      float64
 11  generalRemarks                 0 non-null      float64
 12  value                          1000 non-null   in